In [4]:
import requests
import yaml
with open('../config.yaml', 'r') as file:
    config = yaml.safe_load(file)

# Access the API key
api_key = config['elsevier_api']['api_key']
journal_name = 'Transportation Research Part C: Emerging Technologies'
url = f'https://api.elsevier.com/content/search/scopus?query=SRCTITLE("{journal_name}")&apiKey={api_key}'

response = requests.get(url)
if response.status_code == 200:
    data = response.json()
    articles = data['search-results']['entry']
    for article in articles:
        print(article['dc:title'])
else:
    print("Error:", response.status_code)


An optimization approach for the terminal airspace scheduling problem
Unraveling stochastic fundamental diagrams with empirical knowledge: Modeling, limitations, and future directions
Multimodal traffic assignment considering heterogeneous demand and modular operation of shared autonomous vehicles
Towards dynamic flight separation in final approach: A hybrid attention-based deep learning framework for long-term spatiotemporal wake vortex prediction
Large scale multi-GPU based parallel traffic simulation for accelerated traffic assignment and propagation
A conflict risk graph approach to modeling spatio-temporal dynamics of intersection safety
Individual response prediction and personalized guidance strategy optimization in urban rail transit networks
Predicting flight arrival times with deep learning: A strategy for minimizing potential conflicts in gate assignment
A scalable macro–micro approach for cooperative platoon merging in mixed traffic flows
Reliability modeling for perception